#### Get countries that share species stewardship

In [1]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer
import json

/Users/tamarahuete/opt/anaconda3/envs/arcgis/lib/python3.8/site-packages/arcgis/features/_data/geodataset/utils.py:15: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime,


In [2]:
gis = GIS(profile = "half_earth_profile")

In [132]:
import re
def create_strict_reg_exp(to_search):
    try:
        reg_exp = f"^{to_search}$"
        #logging.info(f"regular expression is: {reg_exp}")
    except:
        print("There was a problem with the string.")
    return reg_exp

In [4]:
def findItemGetID(csvName, gis):
    try:
        searched_item = gis.content.search(csvName, item_type = "Feature Layer")
        for i in searched_item:
            reg_exp = create_strict_reg_exp(csvName)
            if re.search(reg_exp, i.title)!= None:    
                #logging.info(f"{csvName} has the id: {i.id}")
                return i.id
    except:
        print("There was a problem finding the item")

In [5]:
def getFlayerFromID(item_id):
    item = gis.content.get(item_id)
    flayer = item.layers[0]
    return flayer

In [10]:
import pandas as pd
def getSDFfromFlayer(flayer):
    sdf = pd.DataFrame.spatial.from_layer(flayer)
    return sdf

### Fetch species and gadm table and convert to sdf

In [43]:
#### Get the ID of the table.
fl_id  = findItemGetID('Terrestrial_vertebrates_by_country_20200617_stewardship',gis)
fl_gadm= findItemGetID("gadm_centroid", gis)

In [44]:
fl =getFlayerFromID(fl_id)
fl_g = getFlayerFromID(fl_gadm)

In [45]:
sdf = getSDFfromFlayer(fl)
gadm = getSDFfromFlayer(fl_g)

/Users/tamarahuete/opt/anaconda3/envs/arcgis/lib/python3.8/site-packages/arcgis/features/layer.py:2044: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  "esriFieldTypeDate" : pd.datetime,


In [125]:
#sdf.head()

In [127]:
country_list = list(sdf['countryname'].unique())
#len(country_list)) ## 249 countries

In [ ]:
len(gadm['NAME_0']) #252 This is the table of gadm centroid where the countries for the NRC are coming from

GADM table has more countries that the species table

In [56]:
#Missing countries:
list1= list(gadm['NAME_0'])
list2=list(sdf['countryname'].unique())
list(set(list1).difference(list2))

['Bouvet Island', 'Antarctica', 'Åland', 'Western Sahara', 'Cocos Islands']

#### Create a matrix with the species shared by each country. Next step is done with the above_and_below_countries.ipynb
This code is very inneficient.  I was hoping to find something like the scipy.spatial.distance but for count data. Couldn't find and had to do this looping approach. Can probably be fixed
```
library(reshape2)
library(dplyr)
d = read.csv("NRC_species_data_20200817.csv")
head(d)
d_wide =d %>% 
  select(species, iso3) %>% 
  dcast(iso3~species, fun.aggregate = length)
clean_mat = as.matrix(d_wide[,-1])
dimnames(clean_mat)[[1]] = as.character(d_wide$iso3)
df1.mt <- as.matrix(t(clean_mat))
df1.adj <- t(df1.mt) %*% df1.mt
diag(df1.adj)=0
```


In [122]:
### Create empty matrix and make 1 by 1 comparisons.
### Loop over country 1 species list and intersect with country 2 species list
### Could not find a function like the distance functions that would give me counts, so I had to do this very innefificient approach

#super ineficient code 11:44 - 12:15
import numpy
country_list = list(sdf['countryname'].unique())

# create empty matrix 249 x 249
mat = numpy.zeros(shape=(249,249))

for a in range(0,249):
    for b in range(0,249):
        mat[a][b] =len(set(list(sdf[sdf['countryname']==country_list[a]]['species'].unique()) ).intersection(list(sdf[sdf['countryname']==country_list[b]]['species'].unique())))

df =pd.DataFrame(
    mat,
    columns = country_list,
    index = country_list
)

In [126]:
df.head()

,Afghanistan,Akrotiri and Dhekelia,Albania,Algeria,Andorra,American Samoa,Angola,Anguilla,Argentina,Antigua and Barbuda,...,United States Minor Outlying Islands,Uzbekistan,Venezuela,Vanuatu,Vietnam,"Virgin Islands, U.S.",Wallis and Futuna,Yemen,Zambia,Zimbabwe
Afghanistan,664.0,93.0,187.0,202.0,147.0,1.0,64.0,11.0,14.0,10.0,...,4.0,381.0,17.0,4.0,154.0,16.0,0.0,128.0,63.0,63.0
Akrotiri and Dhekelia,93.0,163.0,122.0,120.0,99.0,1.0,22.0,5.0,10.0,6.0,...,1.0,93.0,8.0,3.0,39.0,11.0,0.0,39.0,24.0,22.0
Albania,187.0,122.0,398.0,215.0,239.0,1.0,68.0,15.0,18.0,15.0,...,3.0,197.0,20.0,4.0,91.0,21.0,0.0,71.0,69.0,64.0
Algeria,202.0,120.0,215.0,516.0,202.0,2.0,113.0,19.0,24.0,21.0,...,3.0,190.0,29.0,6.0,102.0,27.0,1.0,152.0,112.0,106.0
Andorra,147.0,99.0,239.0,202.0,305.0,2.0,61.0,15.0,19.0,14.0,...,2.0,152.0,21.0,5.0,80.0,20.0,1.0,59.0,59.0,55.0


In [166]:
df.reset_index(inplace=True)
df= df.rename(columns={'Unnamed: 0':'Country_index'})
# Save local copy
df3.to_csv('stewardship_matrix.csv',index=False)

#### Upload as csv to arcgis and share with Vizzuality dev

In [167]:
group_id='46783c33c294485abe83f5e899c2f21f' #Vizzuality Development Group

In [168]:
csv_file = 'stewardship_matrix.csv'
csv_item = gis.content.add({}, csv_file)

In [169]:
display(csv_item)

<Item title:"stewardship_matrix" type:CSV owner:tamara.huete_eowilson>

In [170]:
csv_lyr = csv_item.publish(None)

In [173]:
csv_lyr.share(everyone=False, org=False, groups=[group_id], allow_members_to_edit=False)

{'results': [{'itemId': '61cdbe0a92774f98b1ddaf3d3d3a1537',
   'success': True,
   'notSharedWith': ['']}]}

In [174]:
csv_lyr.shared_with() ## Not working...?

TypeError: 'dict' object is not callable